In [1]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import io
from PIL import Image
import glob
import os
import pandas as pd
import matplotlib
matplotlib.style.use('dark_background')
import re
import seaborn as sns
import sys
sys.path.append('E:/')
from function import *
import h5py



In [2]:
pc = 'PC'
if pc == 'PC':
    basevol = 'E:/sophia'
else:
    basevol = '/Volumes/SSD_various/'
    
basepath = basevol + '/shark_data/'
output_video_folder = basepath + "track_example_videos/"
npz_directory = basepath + 'data/' ##### Change with your path 
figures_path = basepath + "figures/"

files_list = np.sort(glob.glob(npz_directory + "*0253*"))
filename_with_extension = os.path.basename(files_list[0])
filename = os.path.splitext(filename_with_extension)[0]
base_filename = filename.rsplit('_', 1)[0]

video_filename = basepath + base_filename + ".MP4"
settings_file_path = basepath + base_filename + ".settings"
### Get settings info
# cm_per_pixel = get_trex_settings_value(settings_file_path,'cm_per_pixel')

# Get original file size info
image_path = basepath + 'average_' + base_filename + ".png"

image = cv.imread(image_path)
height, width = image.shape[:2]
print(f"Dimensions of the image: Width = {width}, Height = {height}")

moving_average_window = 12
fs = 50
cm_per_pixel = 1
keys_to_extract = ["frame_segments","SPEED#wcentroid", "ANGLE","frame","time","poseX0",'poseY0',"poseX1",'poseY1',"poseX2",'poseY2',"poseX3",'poseY3',"poseX4",'poseY4',"poseX5",'poseY5',"poseX6",'poseY6',"poseX7",'poseY7',"poseX8",'poseY8']  # Add the specific keys you want to extract from npz file

# cm_per_pixel  = 0.00390
# video_height_cm = height*cm_per_pixel



### Get all filedata names
fish_data = read_files(npz_directory,'npz',include_posture= True,prefix_filter = base_filename)

npz_files = [file for files in fish_data.values() for file in files]
print(npz_files)
# 20240303_072831125_DJI_0266
### Defined what is schooling data and what is shark data
fschool_files = [f for f in npz_files if 'fschool_posture' in f]
shark_files = [f for f in npz_files if 'fschool' not in f]

full_paths_sharks = [os.path.join(npz_directory, file) for file in shark_files]
full_paths_schools = [os.path.join(npz_directory, file) for file in fschool_files]

Dimensions of the image: Width = 5120, Height = 2700
['sequence_20240305_063615511_DJI_0253_fish2.npz', 'sequence_20240305_063615511_DJI_0253_fish1.npz', 'sequence_20240305_063615511_DJI_0253_fish0.npz']


In [12]:
# ### Load outline data
# data, screen, input_shape, output_shape = load_and_preprocess_outline_data(full_paths_schools,fs = fs)


In [13]:
# ### Load shark data
# sharks_df= read_npz_files(npz_directory,shark_files,keys_to_extract, cm_per_pixel,moving_average_window,smooth=True)

In [3]:
# Main processing loop for multiple files
pc = 'PC'
if pc == 'PC':
    basevol = 'E:/sophia'
else:
    basevol = '/Volumes/SSD_various/'
    
basepath = basevol + '/shark_data/'
output_folder = basepath + "processed_data/"
npz_directory = basepath + 'data/'
figures_path = basepath + "figures/"
output_folder = basepath + "processed_data/"
npz_directory = basepath + 'data/'
figures_path = basepath + "figures/"

# List of trials to process
# file_filters = ["*0262*", "*0257*"]
file_filters = ["*DJI_0253*"]

for file_filter in file_filters:
    files_list = np.sort(glob.glob(npz_directory + file_filter))
    
    # Check if files_list is empty
    if not len(files_list):  # or simply: if not files_list
        print(f"No files found for {file_filter}")
        continue

    filename_with_extension = os.path.basename(files_list[0])
    filename = os.path.splitext(filename_with_extension)[0]
    base_filename = filename.rsplit('_', 1)[0]

    video_filename = basepath + base_filename + ".MP4"
    settings_file_path = basepath + base_filename + ".settings"
    image_path = basepath + 'average_' + base_filename + ".png"

    image = cv.imread(image_path)
    height, width = image.shape[:2]
    print(f"Dimensions of the image: Width = {width}, Height = {height}")

    moving_average_window = 12

    fs = 50
    cm_per_pixel = 1

    keys_to_extract = ["frame_segments","SPEED#wcentroid", "X#pcentroid", "Y#pcentroid", "ANGLE","frame","time","poseX0",'poseY0',"poseX1",'poseY1',"poseX2",'poseY2',"poseX3",'poseY3',"poseX4",'poseY4',"poseX5",'poseY5',"poseX6",'poseY6',"poseX7",'poseY7',"poseX8",'poseY8']

    fish_data = read_files(npz_directory,'npz',include_posture=True, prefix_filter=base_filename)
    npz_files = [file for files in fish_data.values() for file in files]

    fschool_files = [f for f in npz_files if 'fschool_posture' in f]
    shark_files = [f for f in npz_files if 'fschool' not in f]

    full_paths_sharks = [os.path.join(npz_directory, file) for file in shark_files]
    full_paths_schools = [os.path.join(npz_directory, file) for file in fschool_files]

    print(f"Processing shark files for {base_filename}: {full_paths_sharks}")
    print(f"Processing school files for {base_filename}: {full_paths_schools}")

    # Process and save shark files
    if full_paths_sharks:
        sharks_df = read_npz_files(npz_directory, shark_files, base_filename, keys_to_extract, cm_per_pixel, moving_average_window, smooth=True)
        ### Add midpoint
        sharks_df['midpoint_x'] = (sharks_df['lxfinx'] + sharks_df['rxfinx']) / 2
        sharks_df['midpoint_y'] = (sharks_df['lxfiny'] + sharks_df['rxfiny']) / 2
        
        # Save sharks_df to HDF5
        sharks_output_file = os.path.join(output_folder, f"{base_filename}_sharks.h5")
        sharks_df.to_hdf(sharks_output_file, key='sharks', mode='w')
        print(f"Shark data saved to {sharks_output_file}")

    
    ### Comment this if you don't want school outline data
    
    # Process and save school files
    if full_paths_schools:
        school_data, school_screen, school_input_shape, school_output_shape = load_and_preprocess_outline_data(full_paths_schools)
        school_output_file = os.path.join(output_folder, f"{base_filename}_school.h5")
        # save_school_data_to_h5(school_data, school_screen, school_input_shape, school_output_shape, school_output_file)
        np.savez(output_folder + base_filename + '_school_data.npz', school_data=school_data, school_screen=school_screen, school_output_shape=school_output_shape)
        print(f"Outline data saved to {output_folder+ base_filename + '_school_data.npz'}")

Dimensions of the image: Width = 5120, Height = 2700
Processing shark files for sequence_20240305_063615511_DJI_0253: ['E:/sophia/shark_data/data/sequence_20240305_063615511_DJI_0253_fish2.npz', 'E:/sophia/shark_data/data/sequence_20240305_063615511_DJI_0253_fish1.npz', 'E:/sophia/shark_data/data/sequence_20240305_063615511_DJI_0253_fish0.npz']
Processing school files for sequence_20240305_063615511_DJI_0253: []
Processing file: sequence_20240305_063615511_DJI_0253_fish0.npz
Processing file: sequence_20240305_063615511_DJI_0253_fish1.npz
Processing file: sequence_20240305_063615511_DJI_0253_fish2.npz
Shark data saved to E:/sophia/shark_data/processed_data/sequence_20240305_063615511_DJI_0253_sharks.h5


In [8]:
# data = np.load(npz_directory + shark_files[0])

In [9]:
# list(data.keys())

In [ ]:
# def display_dict_structure(data_dict, indent=0, max_items=10):
#     indent_str = '  ' * indent  # Indentation for better readability
#     for key, value in data_dict.items():
#         if isinstance(value, dict):
#             print(f"{indent_str}{key}: (dict) -> {len(value)} keys")
#             if len(value) > max_items:  # If there are too many keys, summarize
#                 print(f"{indent_str}  ... and {len(value) - max_items} more keys")
#                 keys_to_show = list(value.keys())[:max_items]
#             else:
#                 keys_to_show = value.keys()
#             for subkey in keys_to_show:
#                 print(f"{indent_str}  {subkey}:")
#                 display_dict_structure({subkey: value[subkey]}, indent + 2)  # Show fewer items

#         elif isinstance(value, list):
#             print(f"{indent_str}{key}: (list) -> {len(value)} items")
#             if len(value) > max_items:
#                 print(f"{indent_str}  Showing first {max_items} items:")
#                 value = value[:max_items]
#             for item in value:
#                 display_dict_structure({key: item}, indent + 1)

#         elif isinstance(value, np.ndarray):
#             print(f"{indent_str}{key}: (numpy array) -> shape: {value.shape}, dtype: {value.dtype}")

#         elif isinstance(value, (str, int, float)):
#             print(f"{indent_str}{key}: ({type(value).__name__}) -> {value}")

#         else:
#             print(f"{indent_str}{key}: ({type(value).__name__}) -> Cannot determine exact details")

# # Usage example
# display_dict_structure(school_data, max_items=5)